In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('TkAgg')

In [37]:
# 获取数据集
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [38]:
# data loaders
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [44]:
print("Training data: ", training_data.data.shape)
print("Test data: ", test_data.data.shape)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Training data:  torch.Size([60000, 28, 28])
Test data:  torch.Size([10000, 28, 28])
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [45]:
# 训练方式 CPU/CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [46]:
# 定义模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(64 * 12 * 12, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [47]:
# 实例
model = CNN().to(device)
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=9216, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [48]:
# 损失函数 和 优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)

In [49]:
# 训练和测试函数
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [50]:
# 训练
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.337576  [    0/60000]
loss: 0.150446  [ 6400/60000]
loss: 0.138385  [12800/60000]
loss: 0.064525  [19200/60000]
loss: 0.037840  [25600/60000]
loss: 0.127241  [32000/60000]
loss: 0.115590  [38400/60000]
loss: 0.130855  [44800/60000]
loss: 0.234214  [51200/60000]
loss: 0.044361  [57600/60000]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.053075 

Epoch 2
-------------------------------
loss: 0.021604  [    0/60000]
loss: 0.120579  [ 6400/60000]
loss: 0.028559  [12800/60000]
loss: 0.010489  [19200/60000]
loss: 0.006350  [25600/60000]
loss: 0.049261  [32000/60000]
loss: 0.069367  [38400/60000]
loss: 0.053115  [44800/60000]
loss: 0.091637  [51200/60000]
loss: 0.035630  [57600/60000]
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.038074 

Epoch 3
-------------------------------
loss: 0.004156  [    0/60000]
loss: 0.081438  [ 6400/60000]
loss: 0.048517  [12800/60000]
loss: 0.051884  [19200/60000]
loss: 0.033000  [25600/60000]
loss: 0.012052  [32000/600

In [72]:
# 保存模型
torch.save(model.state_dict(), "./data/MNIST/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [52]:
# 加载模型
model = CNN()
model.load_state_dict(torch.load("./data/MNIST/model.pth"))

<All keys matched successfully>

In [71]:
# 将训练好的模型用于预测
num = 5
x, y = test_data[num][0], test_data[num][1]
x = x.unsqueeze(dim=0)
with torch.no_grad():
    pred = model(x)
    pred = pred.argmax(1).type(torch.int).item()
    print("pred: ", pred)
    print("target: ", y)
plt.imshow(x[0][0], cmap="Greys_r")
plt.show()

pred:  1
target:  1
